# Using Twitcher with Keycloak

Setup a [Keycloak](https://www.keycloak.org/) service with 
[client credentials grant type](https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow).
You can use this [Ansible playbook](https://github.com/bird-house/ansible-keycloak-playbook). 
The Keycloak service is running on: http://localhost:8080

Configure the twitcher as described in the keycloak example of the [twitcher tutorial](https://twitcher.readthedocs.io/en/latest/tutorial.html) and register the [Emu](https://github.com/bird-house/emu) WPS.


In [ ]:
# disable ssl warnings
import urllib3
urllib3.disable_warnings()

### Keycloak client

In [ ]:
keycloak_url = 'http://localhost:8080'
token_endpoint = '/auth/realms/demo/protocol/openid-connect/token'
client_id = 'demo'
client_secret = 'c083d72c-a262-40b1-ad51-326f6977d74b'

### Get OAuth access token from Keycloak
scope=compute

In [ ]:
token_url = "{}{}".format(keycloak_url, token_endpoint)
token_url

In [ ]:
import os
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(
    token_url,
    scope='compute',
    client_id=client_id,
    client_secret=client_secret,
    include_client_id=True,
    verify=False)
token

In [ ]:
token['access_token']

### Execute WPS Process with access token

In [ ]:
base_url = 'http://localhost:8000'
url = "{}/ows/proxy/emu?service=WPS&version=1.0.0&request=Execute&identifier=chomsky".format(base_url)
url

In [ ]:
import requests
headers = {'Authorization': 'Bearer {}'.format(token['access_token'])}

resp = requests.get(url, headers=headers, verify=False)
resp.ok

In [ ]:
'ProcessSucceeded' in resp.text